In [1]:
!pip install datasets

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import math
import numpy as np

from sklearn.metrics import f1_score, accuracy_score

In [3]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-large")

if tokenizer.bos_token is None:
    tokenizer.bos_token = '<bos>'
    tokenizer.bos_token_id = tokenizer.convert_tokens_to_ids('<bos>')

if tokenizer.eos_token is None:
    tokenizer.eos_token = '<eos>'
    tokenizer.eos_token_id = tokenizer.convert_tokens_to_ids('<eos>')

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [4]:
from datasets import load_dataset
dataset = load_dataset("nyu-mll/glue", "sst2")

train = dataset["train"]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [5]:
train_text = train['sentence']
train_label = train['label']


In [6]:
train_text, test_text, train_label, test_label = train_test_split(train_text, train_label, test_size=0.1, random_state=42)

In [7]:
train_text, val_text, train_label, val_label = train_test_split(train_text, train_label, test_size=0.1, random_state=42)

In [8]:
print(len(train_text))
print(len(train_label))

print(len(test_text))
print(len(test_label))

print(len(val_text))
print(len(val_label))

54552
54552
6735
6735
6062
6062


In [9]:
class SentimentDataset(Dataset): 
  def __init__(self, text, label, tokenizer, max_length=64): 
    self.text = text
    self.label = label 
    self.tokenizer = tokenizer 
    self.max_length = max_length

  def __len__(self): 
    return len(self.label) 

  def __getitem__(self, idx): 
    text_embedding = self.tokenizer.encode_plus(self.text[idx], return_tensors='pt', padding='max_length', max_length=self.max_length, truncation=True)
    label = torch.tensor(self.label[idx], dtype=torch.long) 
    return text_embedding['input_ids'].squeeze(), label
  

train_dataset = SentimentDataset(train_text, train_label, tokenizer)
test_dataset = SentimentDataset(test_text, test_label, tokenizer)
val_dataset = SentimentDataset(val_text, val_label, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [10]:
class Encoder(nn.Module): 
  def __init__(self, vocab_size, max_seq_length, embedding_dim, hidden_dim, dropout): 
    super(Encoder, self).__init__()

    self.vocab_size = vocab_size
    self.max_seq_length = max_seq_length 
    self.embedding_dim = embedding_dim 
    self.hidden_dim = hidden_dim 
    self.dropout = dropout 

    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)

    self.dropout = nn.Dropout(dropout) 

    self.fc = nn.Sequential(
        nn.Linear(max_seq_length * hidden_dim, hidden_dim),
        nn.ReLU(), 
        nn.Linear(hidden_dim, 2)
    )

  def forward(self, x): 
    x = self.dropout(self.embedding(x))
    x, _ = self.gru(x) 
    x = x.reshape(x.size(0), -1)
    x = self.fc(x) 
    return x 


x = torch.randint(size=(128, 64), low=0, high=100)

net = Encoder(vocab_size=64000, max_seq_length=64, embedding_dim=128, hidden_dim=256, dropout=0.1)
out = net(x)
print(out.shape)




torch.Size([128, 2])


In [11]:
vocab_size = tokenizer.vocab_size
max_seq_length = 64
embedding_dim = 256
hidden_dim = 512
dropout = 0.1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Encoder(vocab_size=vocab_size, max_seq_length=max_seq_length, embedding_dim=embedding_dim, hidden_dim=hidden_dim, dropout=dropout).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
epochs = 30

train_loss = []
val_loss = []

for epoch in range(epochs):
  model.train()
  total_train_loss = 0
  for inputs, labels in tqdm(train_loader):

    inputs = inputs.to(device)
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    total_train_loss += loss.item()
    train_loss.append(loss.item())

  print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {total_train_loss / len(train_loader)}')

  model.eval()
  total_val_loss = 0
  f_1 = 0
  accuracy = 0
  for inputs, labels in tqdm(val_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      total_val_loss += loss.item()

      val_loss.append(loss.item())

      preds = torch.argmax(outputs, dim=-1)
      f_1 += f1_score(labels.cpu(), preds.cpu())
      accuracy += accuracy_score(labels.cpu(), preds.cpu())

  print(f'Epoch {epoch + 1}/{epochs}, Val Loss: {total_val_loss / len(val_loader)}')
  print(f'F1 Score: {f_1 / len(val_loader)}')
  print(f'Accuracy: {accuracy / len(val_loader)}')

torch.save(model.state_dict(), '/kaggle/working/rnn_sen.pth')

100%|██████████| 853/853 [00:36<00:00, 23.38it/s]


Epoch 1/30, Train Loss: 0.5874617492467828


100%|██████████| 95/95 [00:02<00:00, 34.90it/s]


Epoch 1/30, Val Loss: 0.47814940182786236
F1 Score: 0.7882128933121518
Accuracy: 0.77162471395881


100%|██████████| 853/853 [00:35<00:00, 24.30it/s]


...

Epoch 29/30, Val Loss: 0.5553673773219711
F1 Score: 0.9096938096666619
Accuracy: 0.9044765446224257


100%|██████████| 853/853 [00:34<00:00, 24.94it/s]


Epoch 30/30, Train Loss: 0.012443516425639846


100%|██████████| 95/95 [00:02<00:00, 40.48it/s]


Epoch 30/30, Val Loss: 0.5995453168687067
F1 Score: 0.9091647201417161
Accuracy: 0.9014159038901602


In [12]:
model.eval()
with torch.no_grad(): 
    f_1 = 0
    accuracy = 0
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=-1) 
        f_1 += f1_score(labels.cpu(), preds.cpu())
        accuracy += accuracy_score(labels.cpu(), preds.cpu())
        
    print(f'F1 score on test set: {f_1/len(test_loader)}')
    print(f'Accuracy on test set: {accuracy/len(test_loader)}')
        

100%|██████████| 106/106 [00:03<00:00, 34.29it/s]

F1 score on test set: 0.9139370502144327
Accuracy on test set: 0.9031544811320755
